In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import docker
import time
import concurrent.futures
from datetime import datetime
import copy
import math
from collections import defaultdict
import pprint
from collections import namedtuple
from datetime import datetime
import os
import ctypes as ct
import multiprocessing
import shutil

In [2]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
def parse_start_time(start_time_str):
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()

if __name__ == "__main__":
    client = docker.from_env()
    isolated_benchmarking_results = {}

    # Prepare disk files (block until done)
    print("Container started for Disk benchmark (prepare).")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "prepare"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )

    tasks = [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
             "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()

    # Disk cleanup (block until done)
    print("Disk cleanup...")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=200G",
            "--file-num=128",
            "--file-block-size=16K",
            "--file-extra-flags=direct",
            "cleanup"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )
    print("Disk benchmark completed.")
    print("All isolated benchmarks completed.")
    client.close()

Container started for Disk benchmark (prepare).
Container lifetime: 15.562562 seconds
Container on CPU 2 completed and removed.
Memory required:  7824K.


LINPACK benchmark, Double precision.
Machine precision:  15 digits.
Array size 1000 X 1000.
Average rolled and unrolled performance:

    Reps Time(s) DGEFA   DGESL  OVERHEAD    KFLOPS
----------------------------------------------------
      32   0.59  89.62%   0.72%   9.66%  10065705.997
      64   1.18  89.63%   0.72%   9.65%  10067736.426
     128   2.36  89.64%   0.72%   9.64%  10049440.237
     256   4.74  89.66%   0.72%   9.62%  10029262.661
     512   9.46  89.64%   0.72%   9.63%  10046129.670
    1024  18.89  89.64%   0.72%   9.64%  10057630.171


Container lifetime: 37.938103 seconds
Container on CPU 0 completed and removed.
Container lifetime: 141.634784 seconds
Container on CPU 4 completed and removed.
Disk cleanup...
Disk benchmark completed.
All isolated benchmarks completed.


In [44]:
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

affectionate_heisenberg: {'coloc_pair': None, 'workload': 'mem', 'id': '5cafbec533aeced9679f1638221287460c6c40c527149b8f83620a275cb85c98', 'life_time': 15.562562}
sweet_cori: {'coloc_pair': None, 'workload': 'cpu', 'id': '4228d09536300144c759ecbf35590045bc88f5a76331485dfb7d47b07f487553', 'life_time': 37.938103}
serene_wilson: {'coloc_pair': None, 'workload': 'fileio', 'id': '6dbed9e19cb1e89afd210e1d5115d70a404f87b56e1fc483372ff0dcefcb2824', 'life_time': 141.634784}


In [46]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"

print("Container names and IDs:")
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in isolated_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  # Don't process contents of a deleted directory

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    isolated_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final isolated benchmarking results with power consumption:")
pprint.pprint(isolated_benchmarking_results)

Container names and IDs:
affectionate_heisenberg: {'coloc_pair': None, 'workload': 'mem', 'id': '5cafbec533aeced9679f1638221287460c6c40c527149b8f83620a275cb85c98', 'life_time': 15.562562}
sweet_cori: {'coloc_pair': None, 'workload': 'cpu', 'id': '4228d09536300144c759ecbf35590045bc88f5a76331485dfb7d47b07f487553', 'life_time': 37.938103}
serene_wilson: {'coloc_pair': None, 'workload': 'fileio', 'id': '6dbed9e19cb1e89afd210e1d5115d70a404f87b56e1fc483372ff0dcefcb2824', 'life_time': 141.634784}
Keeping power metric: output/4228d0953630/power
Keeping file: output/4228d0953630/power/timeseries.csv
Power mean for output/4228d0953630/power/timeseries.csv: 6.863079727620969
Keeping power metric: output/6dbed9e19cb1/power
Keeping file: output/6dbed9e19cb1/power/timeseries.csv
Power mean for output/6dbed9e19cb1/power/timeseries.csv: 1.15773952624457
Keeping power metric: output/5cafbec533ae/power
Keeping file: output/5cafbec533ae/power/timeseries.csv
Power mean for output/5cafbec533ae/power/timese

In [ ]:
# Using linpack | Run co-located benchmarks in a Docker container on the same core physical core using logical pairs.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    
if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "prepare"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )
    
    
    tasks = [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem"},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
    ]

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem",},
        ]},
        {"MemFileIO": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem",},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
        ]},
        {"FileIOCpu": [
        # {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"},
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},]},
        {"CpuCpu": [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        ]},
        {"MemMem": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},
        ]},
        {"FileIOFileIO": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=200G",
            "--file-num=128",
            "--file-block-size=16K",
            "--file-extra-flags=direct",
            "cleanup"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )

    
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 
    
print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container lifetime: 6.323188 seconds
Container on CPU 2 completed and removed.
Memory required:  7824K.


LINPACK benchmark, Double precision.
Machine precision:  15 digits.
Array size 1000 X 1000.
Average rolled and unrolled performance:

    Reps Time(s) DGEFA   DGESL  OVERHEAD    KFLOPS
----------------------------------------------------
      32   0.59  89.63%   0.71%   9.66%  10075421.692
      64   1.18  89.65%   0.70%   9.64%  10061411.742
     128   2.37  89.67%   0.71%   9.62%  10036595.913
     256   4.72  89.64%   0.71%   9.65%  10067724.619
     512   9.44  89.64%   0.71%   9.65%  10067673.848
    1024  18.85  89.63%   0.70%   9.66%  10083442.033


Container lifetime: 37.848341 seconds
Container on CPU 0 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container lifetime: 6.269741 seconds
Container on CPU 2 completed and removed.
Container 

In [7]:
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

ecstatic_pasteur: {'coloc_pair': None, 'workload': 'mem', 'id': 'fe52bf17554b695520f18f38355a46d41e73036c36cfdcc5742c4ba54b191aeb', 'life_time': 7.257822}
crazy_nightingale: {'coloc_pair': None, 'workload': 'cpu', 'id': '66f0eac1530d6f8c56f4bb9946af437cc72af1bf1e08423398c6c121aee279c6', 'life_time': 38.029981}
trusting_sammet: {'coloc_pair': None, 'workload': 'fileio', 'id': '63db1a729f2eacde7c7d1c80da650bc885892c8aba655a13daefc2956a707bc3', 'life_time': 124.872952}


In [48]:
# Access the watched benchmark containers for runtime and power consumption.
coloc_benchmarking_results_copy = copy.deepcopy(coloc_benchmarking_results)

print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")
# print(benchmarking_results)

Container names and IDs:
recursing_golick: {'coloc_pair': 'CpuMem', 'workload': 'mem', 'id': 'bd42856447c161f3d05f9ff8d053721cb1256b5087bc12b4b8ee2c3ad2bcdba2', 'colocated_runtime': 6.323188}
loving_curie: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'c74ea43a3d7695f17d6031e3235ba608c66973b7c16fe824f348f2cbab52a3f9', 'colocated_runtime': 37.848341}
determined_cohen: {'coloc_pair': 'MemFileIO', 'workload': 'mem', 'id': '4985144b92c99b99e2283ff716b21071c1bea8373742f6bade0d8cb6ad660bc0', 'colocated_runtime': 6.269741}
condescending_swanson: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '374712c895367c870207ebadabb4cc7e7cee0c5e827f707204a22dbf98677605', 'colocated_runtime': 141.888545}
great_shtern: {'coloc_pair': 'FileIOCpu', 'workload': 'cpu', 'id': '934e83a75c8785c687e663da18ad24c7f23916d4d8e243cd6ae980f26bbd75bb', 'colocated_runtime': 38.064636}
hungry_babbage: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': '93224d73bf81f56cff36275b649861800099c523f1c81bf70ca946

In [49]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"


print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in coloc_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    if container_name and container_name in coloc_benchmarking_results:
                        coloc_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final colocated benchmarking results with power consumption:")
pprint.pprint(coloc_benchmarking_results)

Container names and IDs:
recursing_golick: {'coloc_pair': 'CpuMem', 'workload': 'mem', 'id': 'bd42856447c161f3d05f9ff8d053721cb1256b5087bc12b4b8ee2c3ad2bcdba2', 'colocated_runtime': 6.323188}
loving_curie: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'c74ea43a3d7695f17d6031e3235ba608c66973b7c16fe824f348f2cbab52a3f9', 'colocated_runtime': 37.848341}
determined_cohen: {'coloc_pair': 'MemFileIO', 'workload': 'mem', 'id': '4985144b92c99b99e2283ff716b21071c1bea8373742f6bade0d8cb6ad660bc0', 'colocated_runtime': 6.269741}
condescending_swanson: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '374712c895367c870207ebadabb4cc7e7cee0c5e827f707204a22dbf98677605', 'colocated_runtime': 141.888545}
great_shtern: {'coloc_pair': 'FileIOCpu', 'workload': 'cpu', 'id': '934e83a75c8785c687e663da18ad24c7f23916d4d8e243cd6ae980f26bbd75bb', 'colocated_runtime': 38.064636}
hungry_babbage: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': '93224d73bf81f56cff36275b649861800099c523f1c81bf70ca946

In [58]:
def calc_average_slowdown(slowdown_1, slowdown_2):
    average_slowdown = (slowdown_1 + slowdown_2) / 2
    return average_slowdown

def calc_slowdown_factor(isolated_runtime_1, isolated_runtime_2, coloc_runtime_1, coloc_runtime_2, isolated_power_consumption_1, isolated_power_consumption_2, coloc_power_consumption_1, coloc_power_consumption_2):
    slowdown_1 = isolated_runtime_1 / coloc_runtime_1
    slowdown_2 = isolated_runtime_2 / coloc_runtime_2
    power_slowdown_1 = isolated_power_consumption_1 / coloc_power_consumption_1
    power_slowdown_2 = isolated_power_consumption_2 / coloc_power_consumption_2
    average_runtime_slowdown = calc_average_slowdown(slowdown_1, slowdown_2)
    average_power_slowdown = calc_average_slowdown(power_slowdown_1, power_slowdown_2) 
    average_slowdown = (average_runtime_slowdown + average_power_slowdown) / 2

    return average_slowdown, 

# def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2):
#     affinity_score = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
#     return min(1, affinity_score)

def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2,
                        isolated_power_consumption_1, coloc_power_consumption_1,
                        isolated_power_consumption_2, coloc_power_consumption_2):
    # Runtime affinity
    runtime_affinity = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
    # Power affinity
    power_affinity = (isolated_power_consumption_1 + isolated_power_consumption_2) / (coloc_power_consumption_1 + coloc_power_consumption_2)
    # Average both
    affinity_score = (runtime_affinity + power_affinity) / 2
    # return min(1, affinity_score)
    return  affinity_score
    
for name, result in coloc_benchmarking_results.items():
    coloc_workload = result.get('workload')
    # Find the isolated runtime for the same workload
    isolated_runtime = ''
    isolated_power_consumption = ''
    for iso_name, iso_result in isolated_benchmarking_results.items():
        if iso_result.get('workload') == coloc_workload and iso_result.get('coloc_pair') is None:
            isolated_runtime = iso_result.get('life_time')
            isolated_power_consumption = iso_result.get('power')
            break
    result['isolated_runtime'] = isolated_runtime
    result['isolated_power_consumption'] = isolated_power_consumption
    print(f"Colocated benchmark: {coloc_workload}, {result.get('coloc_pair')}, Isolated runtime: {isolated_runtime}, Isolated power consumption: {isolated_power_consumption}")

Colocated benchmark: mem, CpuMem, Isolated runtime: 15.562562, Isolated power consumption: 8.339141243806328
Colocated benchmark: cpu, CpuMem, Isolated runtime: 37.938103, Isolated power consumption: 6.863079727620969
Colocated benchmark: mem, MemFileIO, Isolated runtime: 15.562562, Isolated power consumption: 8.339141243806328
Colocated benchmark: fileio, MemFileIO, Isolated runtime: 141.634784, Isolated power consumption: 1.15773952624457
Colocated benchmark: cpu, FileIOCpu, Isolated runtime: 37.938103, Isolated power consumption: 6.863079727620969
Colocated benchmark: fileio, FileIOCpu, Isolated runtime: 141.634784, Isolated power consumption: 1.15773952624457
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 37.938103, Isolated power consumption: 6.863079727620969
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 37.938103, Isolated power consumption: 6.863079727620969
Colocated benchmark: mem, MemMem, Isolated runtime: 15.562562, Isolated power consumption: 8.339141243806328
C

In [59]:
# Transform benchmarking dict into a summary
coloc_summary = defaultdict(dict)

for name, result in coloc_benchmarking_results.items():
    pair = result['coloc_pair']
    workload = result.get('workload', 'unknown')
    if 'workload_1' not in coloc_summary[pair]:
        coloc_summary[pair]['workload_1'] = workload
        coloc_summary[pair]['colocated_runtime_1'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_1'] = result.get('isolated_runtime')
        coloc_summary[pair]['isolated_power_consumption_1'] = result.get('isolated_power_consumption')
        coloc_summary[pair]['colocated_power_consumption_1'] = result.get('power')
        
    else:
        coloc_summary[pair]['workload_2'] = workload
        coloc_summary[pair]['colocated_runtime_2'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_2'] = result.get('isolated_runtime')
        coloc_summary[pair]['isolated_power_consumption_2'] = result.get('isolated_power_consumption')
        coloc_summary[pair]['colocated_power_consumption_2'] = result.get('power')
        
# calculate and add average_slowdown and affinity_score for each coloc pair
for pair, summary in coloc_summary.items():
    try:
        iso_run1 = float(summary['isolated_runtime_1'])
        iso_run2 = float(summary['isolated_runtime_2'])
        coloc_run1 = float(summary['colocated_runtime_1'])
        coloc_run2 = float(summary['colocated_runtime_2'])
        iso_pow1 = float(summary['isolated_power_consumption_1'])
        iso_pow2 = float(summary['isolated_power_consumption_2'])
        coloc_pow1 = float(summary['colocated_power_consumption_1'])
        coloc_pow2 = float(summary['colocated_power_consumption_2'])
        summary['average_slowdown'] = calc_slowdown_factor(iso_run1, iso_run2, coloc_run1, coloc_run2, iso_pow1, iso_pow2, coloc_pow1, coloc_pow2)
        # summary['affinity_score'] = calc_affinity_score(iso_run1, coloc_run1, iso_run2, coloc_run2, iso_pow1, coloc_pow1, iso_pow2,coloc_pow2)
        summary['affinity_score'] = calc_affinity_score(
    iso_run1, coloc_run1, iso_run2, coloc_run2,
    iso_pow1, coloc_pow1, iso_pow2, coloc_pow2
)
    except Exception as e:
        summary['average_slowdown'] = None
        summary['affinity_score'] = None
        print(f"Could not calculate for pair {pair}: {e}")

coloc_summary = dict(coloc_summary)
pprint.pprint(coloc_summary)

{'CpuCpu': {'affinity_score': 2.0036122853556093,
            'average_slowdown': (2.0036179805893224,),
            'colocated_power_consumption_1': 1.9524228298888033,
            'colocated_power_consumption_2': 1.9594562418272168,
            'colocated_runtime_1': 76.098709,
            'colocated_runtime_2': 76.145715,
            'isolated_power_consumption_1': 6.863079727620969,
            'isolated_power_consumption_2': 6.863079727620969,
            'isolated_runtime_1': 37.938103,
            'isolated_runtime_2': 37.938103,
            'workload_1': 'cpu',
            'workload_2': 'cpu'},
 'CpuMem': {'affinity_score': 0.8686447429143824,
            'average_slowdown': (1.5353500152680195,),
            'colocated_power_consumption_1': 3.4631027917974677,
            'colocated_power_consumption_2': 25.433677579885707,
            'colocated_runtime_1': 6.323188,
            'colocated_runtime_2': 37.848341,
            'isolated_power_consumption_1': 8.339141243806328,
 

In [64]:
# Write affinity score matrix to a CSV file
affinity_score_matrix = pd.DataFrame.from_dict(coloc_summary, orient='index')
affinity_score_matrix = affinity_score_matrix[['workload_1', 'workload_2', 'affinity_score']]

print(affinity_score_matrix)
# Write back to csv
affinity_score_matrix.to_csv('affinity_score_matrix.csv', index=False)

             workload_1 workload_2  affinity_score
CpuMem              mem        cpu        0.868645
MemFileIO           mem     fileio      217.328397
FileIOCpu           cpu     fileio        4.824473
CpuCpu              cpu        cpu        2.003612
MemMem              mem        mem      149.686427
FileIOFileIO     fileio     fileio      180.806754
